<a href="https://colab.research.google.com/github/van-dang/MRI-Cloud/blob/master/RealNeurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the code

In [0]:
# This program solves the Bloch-Torrey equation applied to computational diffusion MRI using 
# the finite element method coupled with the theta-method for the spatial discretization.

# The scope of usage: 
# (1) Single domains, Multilayered structures, manifolds
# (2) Membrane permeability for internal interfaces
#     Artificial permeability at the external interfaces
# (3) pure homogeneous Neumann BCs, (4) pseudo-periodic BCs

# Copyright (C) 2019 Van-Dang Nguyen (vdnguyen@kth.se)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-25

# This demo is maintained by Van-Dang Nguyen
# Please report possible problems to vdnguyen@kth.se

# Setting a working environment with FEniCS

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

gmsh_dir=!which gmsh
if len(gmsh_dir)==0:
  !apt-get install gmsh
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

clear_output()

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files

# Disable warnings
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("rothemain.rothe_utils")
logging.getLogger('UFL').setLevel(logging.WARNING)
logging.getLogger('FFC').setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")


E: Package 'python-software-properties' has no installation candidate
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [60.4 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [801 B]
Get:9 http://ppa.launchpad.net/fenics-packages/fenics/ubuntu bionic InRelease [15.4 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu18

#Load pre-defined functions

In [0]:
exists = os.path.isfile('DmriFemBuitInFunc.py')
isupdate = False
if (exists==False or isupdate==True):
    if isupdate==True:
        !rm DmriFemBuitInFunc.py
    print("Load pre-defined functions from GitHub")
    !wget --quiet https://raw.githubusercontent.com/van-dang/MRI-Cloud/master/DmriFemBuitInFunc.py

from DmriFemBuitInFunc import *

class MRI_parameters():
    def __init__(self):
        # Initialize default parameters
        self.bvalue = None
        self.qvalue = None        
        self.gdir = [1, 0, 0];
        self.stype='PGSE'
        self.nperiod = 0; # number of period for OGSE sequences
    def set_gradient_dir(self, mymesh, g0, g1, g2):
        gdim = mymesh.geometry().dim()
        if gdim==2:
            self.gdir = Point(g0, g1)
            self.gdir /= self.gdir.norm()
            self.g = Expression(("g0","g1"), g0=self.gdir.x(), g1=self.gdir.y(),domain=mymesh, degree=1);
        if gdim==3:
            self.gdir = Point(g0, g1, g2)
            self.gdir /= self.gdir.norm()        
            self.g = Expression(("g0","g1","g2"), g0=self.gdir.x(), g1=self.gdir.y(), g2=self.gdir.z(),domain=mymesh, degree=1);
    def time_profile(self, t):
        ft1, ft2 = 0, 0;
        if self.stype=='PGSE':
            ft1 = 1.0*(t>=0 and t<self.delta) 
            ft2 = -1.0*(t>=self.Delta and t<=self.Delta+self.delta);
        if self.stype=='cos_OGSE':
            ft1 =  cos(2.0*self.nperiod*pi*t/self.delta)*(t>=0 and t<self.delta);
            ft2 = -cos(2.0*self.nperiod*pi*(t-2.*self.delta)/self.delta)*(t>=Delta and t<=self.Delta+self.delta);
        return ft1 + ft2; 
    def itime_profile(self, t): 
        # integrate ft
        ft1, ft2, ft3 = 0, 0, 0;
        if self.stype=='PGSE':
            ft1 = t*(t>=0 and t<self.delta) 
            ft2 = self.delta*(t>=self.delta and t<self.Delta) 
            ft3 = (self.delta - t + self.Delta)*(t>=self.Delta and t<=self.Delta+self.delta) 
        if self.stype=='cos_OGSE':
 	          ft1 = 1.0/2.0*self.delta*sin(2.0*t*self.nperiod*pi/self.delta)/self.nperiod/pi * (t >= 0 and  t <= self.delta)
 	          ft2 = 1.0/2.0*self.delta*(sin(2.0*self.Delta*self.nperiod*pi/self.delta)-sin(2.0*t*self.nperiod*pi/self.delta))/nperiod/pi *(t>=self.Delta and t<=self.Delta+self.delta)
        return ft1 + ft2 + ft3;  
    def convert_b2g(self):
        self.gnorm = 0;
        if (self.stype == 'PGSE'):
            self.gnorm = sqrt(self.bvalue)/sqrt(self.delta*self.delta*(self.Delta-self.delta/3.0));
        if (self.stype == 'cos_OGSE'):
            self.gnorm = sqrt(self.bvalue/(1./8.*self.delta*self.delta*self.delta*(3.-cos(4.*self.nperiod*pi/self.delta*self.Delta))/(self.nperiod*self.nperiod)/(pi*pi)));
        if (self.stype == 'sin_OGSE'):
            self.gnorm = sqrt(self.bvalue/(1./8.*self.delta*self.delta*self.delta*(5.+cos(4.*self.nperiod*pi/self.delta*self.Delta))/(self.nperiod*self.nperiod)/(pi*pi)));
        return self.gnorm
    def convert_g2b(self):
        self.bvalue = 0;
        if (self.stype == 'PGSE'):
            self.bvalue = self.gnorm*self.gnorm*self.delta*self.delta*(self.Delta-self.delta/3.0);
        if (self.stype == 'cos_OGSE'):
            self.bvalue=self.gnorm*self.gnorm*1./8.*self.delta*self.delta*self.delta*(3.-cos(4.*self.nperiod*pi/self.delta*self.Delta))/(self.nperiod*self.nperiod)/(pi*pi);
        if (self.stype == 'sin_OGSE'):
	          self.bvalue=self.gnorm*self.gnorm*1./8.*self.delta*self.delta*self.delta*(5.+cos(4.*self.nperiod*pi/self.delta*self.Delta))/(self.nperiod*self.nperiod)/(pi*pi);
        return self.bvalue
      
    def Apply(self):
        self.T = self.Delta+self.delta;
        if not(self.bvalue==None):
            self.gnorm = self.convert_b2g();
            self.qvalue = convert_g2q(self.gnorm);
        elif not(self.qvalue==None):
            self.gnorm = convert_q2g(self.qvalue);
            self.bvalue = self.convert_g2b();
        elif (self.bvalue==None and self.bvalue==None):
            print("bvalue or qvalue need to be specified.")
            sys.exit()      
          
class MRI_simulation():
    def __init__(self):
          self.nskip = 5;    # Output frequency (for visualization only)
          self.theta = 0.5;  # theta=0.5: midpoint method

    def InitialCondition(self, mydomain):
          if mydomain.gdim==2:
              Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]<eps",eps=1e6, domain=mydomain.mymesh, degree=1);
          if mydomain.gdim==3:
              Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]+x[2]*x[2]<eps",eps=1e6, domain=mydomain.mymesh, degree=1);
          Dirac_Delta = interpolate(Dirac_Delta, mydomain.V);
          u_0 = Function(mydomain.W);
          assign(u_0.sub(0), Dirac_Delta)
          if (mydomain.IsDomainMultiple==True):
              assign(u_0.sub(2), Dirac_Delta)  
          return Dirac_Delta, u_0
        
    def solve(self, mydomain, mri_para, linsolver): 
      
          self.Dirac_Delta, self.u_0 = self.InitialCondition(mydomain)
          
          stepcounter = 0;

          M = MassMatrix(mydomain);

          self.t = 0;
          ft_prev  =  mri_para.time_profile(self.t);
          ift_prev = mri_para.itime_profile(self.t);

          start_time = time.time()
          
          while self.t < mri_para.T + self.k: # Time-stepping loop
              if stepcounter % self.nskip == 0:
                  print('t: %6.2f '%self.t, 'T: %6.2f'%mri_para.T, 'dt: %.1f'%self.k,'gnorm: %e'%mri_para.gnorm,'Completed %3.2f%%'%(float(self.t)/float(mri_para.T+mri_simu.k)*100.0));

              ft = mri_para.time_profile(self.t);
              ift = mri_para.itime_profile(self.t);
              L = ThetaMethodL(ft_prev, ift_prev, mri_para, self, mydomain);
              A = 1/self.k*M + assemble(ThetaMethodF(ft, ift, mri_para, self, mydomain))

              b = assemble(L);
              
              linsolver.solve(A, self.u_0.vector(),b);

              ft_prev  = ft;
              ift_prev = ift;

              self.t += self.k;
              stepcounter += 1;

          elapsed_time = time.time() - start_time
          print("Successfully Completed! Elapsed time: %f seconds"%elapsed_time)

# Working on the mesh

In [0]:
geo_choice = 1

if geo_choice == 1:
      !rm -rf *
      !wget https://raw.githubusercontent.com/van-dang/MRI-Cloud/mesh/fru_M_100383_1D.xml.zip
      !unzip -q fru_M_100383_1D.xml.zip
      mesh = Mesh("fru_M_100383_1D.xml");

if geo_choice == 2:
      !rm -rf *
      spindle_list=['03a_spindle2aFI','03a_spindle6aFI','03b_spindle4aACC','03b_spindle5aACC',
                    '03b_spindle6aACC','03b_spindle7aACC','04b_spindle3aFI','05b_spindle5aFI',
                    '06b_spindle8aACC','07b_spindle9aACC','08a_spindle13aACC','09o_spindle7aFI',
                    '09o_spindle8aFI','10a_spindle18aACC','12a_spindle19aACC','12o_spindle9aFI',
                    '13o_spindle10aFI','15o_spindle12aFI','16o_spindle13aFI','19o_spindle14aFI',
                    '21o_spindle15aFI','23o_spindle16aFI','25o_spindle17aFI','26o_spindle18aFI',
                    '27o_spindle19aFI','28o_spindle20aFI','28o_spindle21aFI','29o_spindle22aFI',
                    '30o_spindle23aFI',
      ];

      pyramidal_list=['02a_pyramidal2aFI','02b_pyramidal1aACC','02b_pyramidal1aFI','03a_pyramidal9aFI',
                      '03b_pyramidal2aACC','03b_pyramidal3aACC','03b_pyramidal3aFI','03b_pyramidal4aFI',
                      '03b_pyramidal9aFI','04a_pyramidal4aACC','04a_pyramidal5aACC','04b_pyramidal5aFI',
                      '04b_pyramidal6aACC','04b_pyramidal6aFI','04b_pyramidal7aACC','05a_pyramidal10aACC',
                      '05a_pyramidal8aACC','05b_pyramidal7aFI','05b_pyramidal8aFI','05b_pyramidal9aACC',
                      '06a_pyramidal11aACC','06b_pyramidal10aFI','06b_pyramidal12aACC','07a_pyramidal13aACC',
                      '07b_pyramidal14aACC','08o_pyramidal11aFI','10a_pyramidal15aACC','11a_pyramidal16aACC',
                      '11o_pyramidal12aFI','17o_pyramidal13aFI','18o_pyramidal14aFI','20o_pyramidal15aFI',
                      '22o_pyramidal16aFI','24o_pyramidal17aFI','25o_pyramidal18aFI','31o_pyramidal19aFI',
      ];

      neuron_id = 14;
      neuron_type = 'pyramidals';

      if neuron_type == 'spindles':
          neuron_list = spindle_list;
      if neuron_type == 'pyramidals':
          neuron_list = pyramidal_list;

      neuron_name = neuron_list[neuron_id];
      print(neuron_name)

      neuron_dir='https://raw.githubusercontent.com/van-dang/NeuronVolumeMeshes/master/'+neuron_type+'/'+neuron_name+'.msh.zip'

      !wget -q $neuron_dir
      !unzip -q $neuron_name''.msh.zip
      !dolfin-convert $neuron_name''.msh $neuron_name''.xml
      !sed -i 's/dim="2"/dim="3"/g' $neuron_name''.xml

      mymesh = Mesh(neuron_name+'.xml');
gdim = mymesh.geometry().dim()
tdim = mymesh.topology().dim()

print('Topological dimension: ',tdim)
print('Geometrical dimension: ',gdim)
print('Mesh size: #vertices: '+str(mymesh.num_vertices()) +', #elements: '+str(mymesh.num_cells()) );

# Solve the Bloch-Torrey equation

In [0]:
mri_simu = MRI_simulation()
mri_para = MRI_parameters()

#################################################################################
#########################  Pre-defined parameters ###############################
mri_para.stype='PGSE'                                # sequence type
mri_para.bvalue = 1000;                              # bvalue
mri_para.delta, mri_para.Delta = 10000, 10000        # time sequence
mri_para.set_gradient_dir(mymesh, 0, 1, 0)           # gradient direction
mri_para.Apply()
mri_simu.k = 500;                                    # time-step size
mri_simu.nskip = 1;                                  # frequency to print ouputs
mydomain = MyDomain(mymesh, mri_para)
mydomain.phase = None
mydomain.PeriodicDir = [0, 0, 0];             # Direction of the periodicity
mydomain.IsDomainPeriodic = False             # Confirm if the mesh if periodic
mydomain.IsDomainMultiple = False             # Confirm if the mesh is multiple
mydomain.kappa = 1e-5                         # Permeability
################################################################################
mydomain.Apply()   # Call Apply before setting the diffusion tensor
################################################################################
# Impose the diffusion coefficient
D0 = 3e-3
mydomain.D = D0;
#################################################################################

linsolver = PETScLUSolver("mumps")
#linsolver = KrylovSolver("bicgstab")
#linsolver.parameters["absolute_tolerance"] = 1e-4
#linsolver.parameters["relative_tolerance"] = 1e-3
#linsolver.parameters["maximum_iterations"] = 1000

mri_simu.solve(mydomain, mri_para, linsolver)

#Post-processing

In [0]:
if mydomain.IsDomainMultiple == True:
    u0r_0, u0i_0, u1r_0, u1i_0 = split(mri_simu.u_0)
    signal0 = assemble(((1-mydomain.phase)*u0r_0)*dx)/assemble((1-mydomain.phase)*mri_simu.Dirac_Delta*dx);
    signal1 = assemble((mydomain.phase*u1r_0)*dx)/assemble(mydomain.phase*mri_simu.Dirac_Delta*dx);
    signal = assemble((mydomain.phase*u1r_0+(1-mydomain.phase)*u0r_0)*dx)/assemble(mri_simu.Dirac_Delta*dx);
    print('Signal on each compartment')
    print('Signal0: %.3e'%signal0)
    print('Signal1: %.3e'%signal1)
    print('b:',mri_para.bvalue, 'Signal: %.3e'%signal,'kappa: %.3e'%mydomain.kappa,', dt: ',mri_simu.k,', hmin: ',mydomain.hmin, ', Free signal: %.3e'%(exp(-mri_para.bvalue*D0)))
    V0 = FunctionSpace(mesh0, mydomain.Ve);
    V1 = FunctionSpace(mesh1, mydomain.Ve);
    u0r_0p = project(u0r_0,V0)
    u1r_0p = project(u1r_0,V1)
    plot(u0r_0p, cmap="coolwarm")
    plot(u1r_0p, cmap="coolwarm")  
else:
    ur, ui = split(mri_simu.u_0)
    signal = assemble(ur*dx)/assemble(mri_simu.Dirac_Delta*dx);
    print('b:',mri_para.bvalue, 'Signal: %.3e'%signal,', dt: ',mri_simu.k,', hmin: ',mydomain.hmin, ', Free signal: %.3e'%(exp(-mri_para.bvalue*D0)))
    V = FunctionSpace(mymesh,mydomain.Ve);
    ur_p = project(ur,V)
    plot(ur_p, cmap="coolwarm")